# Análisis de costos de cursos en el mercado

In [1]:
import pandas as pd
import numpy as np
import os
from hermetrics.levenshtein import Levenshtein

# Rutas de archivos

In [2]:
path = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','existing_data','DIRTY_DATA'))
path_additional_data = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','additional_data'))
path_dwh = os.path.abspath(os.path.join(os.getcwd(),'DATAWAREHOUSE'))
path_ppd = os.path.abspath(os.path.join(os.getcwd(),'CLEANED_DATA'))

In [3]:
# Cargar los archivos de costos de educacion continua, costos de cursos cec y cursos de OC, OEC y CI

df_dim_perfil = pd.read_csv(os.path.join(path_dwh,'datamart_oec','dim_perfil.csv'))
df_dim_curso = pd.read_csv(os.path.join(path_dwh,'datamart_ci','dim_curso.csv'))


### Palabras clave

In [4]:
keyword = ['Python', 'programación web', 'paginas web', 'datos', 'Excel', 'Word', 'power point', 'BI', 'inteligencia de negocios', 'powerBI', 
'big data', 'programación', 'marketing', 'web', 'html', 'css',  'java', 'javascript', 'base de datos', 'redes', 'telecomunicaciones', 'Android', 'IOS', 'ARDUINO'
'Business Intelligence', 
'Desarrollo de software',
'Fibra Óptica',
'Gerencia de sistemas',
'Hackeo Ético',
'Informática Forense',
'Linux', 'WINDOWS'
'Office',
'PHP',
'Project',
'software R', 'Tableu', 'rapidminer', 
'Mysql', 'sql', 'sqlserver', 'Oracle', 'postgresql',
'Seguridad informática',
'Aulas virtuales', 'Moodle', 'software',
'Wordpress', 'joomla', 'xoops', 'drupal'
]

keyword = np.char.lower(np.array(keyword))

In [4]:
lev = Levenshtein()

In [5]:
#Filtrar las areas de Tecnologia y TICs

df_dim_curso_perfil = pd.read_csv(os.path.join(path_dwh,'datamart_oc','dim_curso_perfil.csv'))
df_dim_curso_perfil = df_dim_curso_perfil[df_dim_curso_perfil['area_familia'] == 'TECNOLOGÍAS DE LA INFORMACIÓN Y COMUNICACIÓN']

In [6]:
df_coec = pd.read_csv(os.path.join(path_ppd,'costos_cursos_educacion_continua.csv'))
df_ccce = pd.read_csv(os.path.join(path_ppd,'costos_de_cursos_cec_epn.csv'))
df_coec = pd.concat([df_coec,df_ccce])

In [7]:
data= df_dim_curso_perfil.groupby(['curso_perfil','modalidad']).count().reset_index()
data = data[['curso_perfil','modalidad']]
data['curso_perfil'] = 

In [9]:
arr = []

for i in range(len(df_coec['curso'])):
    # Similaridad 
    sub_arr = []
    for j in range(len(data['curso_perfil'])):
        curso_fuente = df_coec['curso'].iloc[i]
        curso_comparar = data['curso_perfil'].iloc[j]
        similary = lev.similarity(curso_fuente,curso_comparar)
        aux_similary=0
        if curso_comparar in curso_fuente :
            similary = 1.0
        else:
            arr_curso = curso_comparar.split(' ')
            for word in arr_curso:
                if word in curso_fuente:
                    aux_similary+=1/len(arr_curso)
        if aux_similary > similary:
            similary = aux_similary
        sub_arr.append(similary)
    arr.append(sub_arr)


In [25]:
res = pd.DataFrame(data=arr,index=df_coec['curso'],columns=data['curso_perfil'])
res['EXCEL BÁSICO'].max()
res[res['EXCEL BÁSICO']==0.5]

C:\Users\sjime\AppData\Local\Temp\ipykernel_17520\778363828.py:3: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  res[res['EXCEL BÁSICO']==0.5]


ValueError: cannot reindex on an axis with duplicate labels

In [90]:
res.max()

curso_perfil
3D STUDIO MAX                                                                      0.074074
3DS MAX + VRAY - VISUALIZACIÓN ARQUITECTÓNICA                                      0.142857
3DS MAX +VRAY -VISUALIZACIÓN ARQUITECTÓNICA                                        0.093023
ADMINISTRACIÓN DE LA PLATAFORMA MOODLE                                             0.086957
ADMINISTRACIÓN DE LA PLATAFORMA MOODLE                                             0.086957
                                                                                     ...   
WATERCAD & SEWERCAD -DISEÑO DE REDES DE AGUA POTABLE Y ALCANTARILLADO SANITARIO    0.088710
WORD AVANZADO                                                                      0.058824
WORD BASICO                                                                        0.062500
WORD INTERMEDIO                                                                    0.047619
WORDPRESS                                                          

In [91]:
#res['3D STUDIO MAX'][res['3D STUDIO MAX']==res['3D STUDIO MAX'].max()]
res[res.max()]

KeyError: "None of [Float64Index([0.07407407407407407, 0.14285714285714285, 0.09302325581395354,\n              0.08695652173913049, 0.08695652173913049, 0.08695652173913049,\n              0.07894736842105265, 0.07894736842105265, 0.09302325581395354,\n              0.04761904761904767,\n              ...\n              0.10810810810810811, 0.07692307692307687, 0.16666666666666666,\n                              0.2, 0.09677419354838712, 0.08870967741935487,\n              0.05882352941176472,              0.0625, 0.04761904761904767,\n                              0.0],\n             dtype='float64', name='curso_perfil', length=356)] are in the [columns]"

In [ ]:




# Realizar un procesado de texto en donde se contenga los nombres de los cursos por modalidad en caso de OC el resto de cursos solo se tomará un promedio general estimado de estos cursos

# Procesado de texto:
#   Aplicar a lista de las palabras clave de tecnologia
#   Validar si un curso es igual a la lista de costos de cursos
#   Buscar simetría en el nombre de los cursos en el sitio web y el nombre de los cursos de las listas de costos
#   Revisará los nombre de todos los cursos y determinará las palabras sin palabras_de_parada que se repitan más para obtener palabras clave en cuenta a cursos para buscarlas